In [1]:
import tensorflow as tf
import numpy as np

import model.cnn as cnn
import model.rnn as rnn
import model.classifier as classifier

import data.ops

import time
import matplotlib.pyplot as plt
%matplotlib inline

# Model definition

In [2]:
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(32, [], name='batch_size')    
cnn_params = {
    'out_dims' : [10, 10, 10],
    'kernel_sizes' : 32,
    'pool_sizes' : 2
}
rnn_params = {
    'rnn_sizes' : [10],
    'time_steps' : 100
}
fc_params = {
    'fc_sizes' : [30, 20, 10]
}

input_op, seq_len, label = data.ops.get_batch_producer(
    batch_size=batch_size, path='./data/train.TFRecord')

c = cnn.get_output(seq_len=seq_len, input_op=input_op, **cnn_params)
r = rnn.get_model(batch_size=batch_size, seq_len=seq_len, input_op=c, **rnn_params)
logits, pred = classifier.get_logits_and_pred(input_op=r.last_output, **fc_params)



CNN--cnn10x32-10x32-10x32
Tensor("CNN/Conv1/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)
Tensor("CNN/Conv2/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)
Tensor("CNN/Conv3/MaxPool2D/MaxPool:0", shape=(?, ?, 1, 10), dtype=float32)

RNN--rnn--steps100--sizes10
LSTMStateTuple(c=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_2:0' shape=(?, 10) dtype=float32>, h=<tf.Tensor 'RNN/LSTM/dynamic_wrapper/rnn/while/Exit_3:0' shape=(?, 10) dtype=float32>)

FC--fc30-20-10
Tensor("classifier/hidden_layer0/fully_connected/Relu:0", shape=(?, 30), dtype=float32)
Tensor("classifier/hidden_layer1/fully_connected/Relu:0", shape=(?, 20), dtype=float32)
Tensor("classifier/hidden_layer2/fully_connected/Relu:0", shape=(?, 10), dtype=float32)
Tensor("classifier/logits/BiasAdd:0", shape=(?, 4), dtype=float32)
Tensor("classifier/predictions:0", shape=(?, 4), dtype=float32)


# Time measure

convenience function

In [3]:
def measure_time(op, feed_dict={}, n_times=10):
    with tf.Session() as sess:
        print('Sess started')
        coord = tf.train.Coordinator()
        tf.global_variables_initializer().run()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        init_state = sess.run(r.zero_state)
        rnn_feed = {r.init_state : init_state}
        feed_dict.update(rnn_feed)
        print('Evaluating')
        for _ in range(n_times):
            t = time.time()
            test_output = sess.run(op, feed_dict)
            print(test_output, 'Eval time:', time.time() - t)
            
        print('Closing threads')
        coord.request_stop()
        coord.join(threads)

        return test_output

# Evaluation

## **Confusion matrix**

## **Accuracy operator**

In [4]:
with tf.name_scope('evaluation'):
    with tf.name_scope('one_hot_encoding'):
        y_oh = tf.cast(tf.equal(
            logits, tf.reduce_max(logits, axis=1)[:, None]), tf.float32)[..., None]

        label_oh = tf.one_hot(label, depth=4)[..., None]
    with tf.name_scope('confusion_matrix'):
        conf_op = tf.reduce_sum(tf.transpose(y_oh, perm=[0, 2, 1]) * label_oh,
            axis=0, name='result')

    with tf.name_scope('accuracy'):
        y_tot = tf.reduce_sum(conf_op, axis=0, name='label_class_sum')
        label_tot = tf.reduce_sum(conf_op, axis=1, name='pred_class_sum')
        correct_op = tf.diag_part(conf_op, name='correct_class_sum')
        eps = tf.constant([1e-10] * 4, name='eps')
        acc_op = tf.reduce_mean(2*correct_op / (y_tot + label_tot + eps), name='result')

In [14]:
conf_op

<tf.Tensor 'evaluation/confusion_matrix/result:0' shape=(4, 4) dtype=float32>

In [5]:
with tf.Session() as sess:
    print('Session started')
    print(conf_op.eval({logits:[[.1, .2, .3, .4], [.1, .2, .3, .4]], label:[3, 0]}))
    print(sess.run([acc_op], {logits:[[.1, .2, .3, .4], [.1, .2, .3, .4]], label:[3, 0]}))

Session started
[[ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[0.16666667]


In [6]:
#measure_time(acc_op, n_times=3)

# Sparse, weighted softmax loss

In [7]:
class_hist = np.load('./data/class_hist.npy')
with tf.name_scope('loss'):
    weight = tf.constant(1 - np.sqrt(class_hist/class_hist.sum()), name='weights')
    weight = tf.gather(weight, label, name='weight_selector')
    loss = tf.losses.sparse_softmax_cross_entropy(label, logits, weight, scope='weighted_loss')
    unweighted_loss = tf.losses.sparse_softmax_cross_entropy(label, logits, scope='unweighted_loss')
class_hist, weight

(array([5154,  771, 2557,   46]),
 <tf.Tensor 'loss/weight_selector:0' shape=(?,) dtype=float64>)

In [8]:
# measure_time([loss, unweighted_loss])

# Train operator

In [9]:
with tf.name_scope('train'):
    learning_rate = tf.Variable(initial_value=.05, trainable=False, name='learning_rate')
    global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
    grad_clip = tf.Variable(initial_value=3., trainable=False, name='grad_clip')
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss)
    with tf.name_scope('gradient_clipping'):
        capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) 
                      for grad, var in gvs]
        
    opt = optimizer.apply_gradients(capped_gvs, global_step)

In [10]:
[v.name for v in tf.trainable_variables()]


['CNN/Conv1/Conv_dim10_ker32_pool2/weights:0',
 'CNN/Conv1/Conv_dim10_ker32_pool2/BatchNorm/beta:0',
 'CNN/Conv2/Conv_dim10_ker32_pool2/weights:0',
 'CNN/Conv2/Conv_dim10_ker32_pool2/BatchNorm/beta:0',
 'CNN/Conv3/Conv_dim10_ker32_pool2/weights:0',
 'CNN/Conv3/Conv_dim10_ker32_pool2/BatchNorm/beta:0',
 'RNN/LSTM/dynamic_wrapper/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights:0',
 'RNN/LSTM/dynamic_wrapper/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/biases:0',
 'classifier/hidden_layer0/fully_connected/weights:0',
 'classifier/hidden_layer0/fully_connected/biases:0',
 'classifier/hidden_layer1/fully_connected/weights:0',
 'classifier/hidden_layer1/fully_connected/biases:0',
 'classifier/hidden_layer2/fully_connected/weights:0',
 'classifier/hidden_layer2/fully_connected/biases:0',
 'classifier/logits/weights:0',
 'classifier/logits/biases:0']

In [11]:
train_writer = tf.summary.FileWriter('/tmp/model/hist/', graph=tf.get_default_graph())
for v in tf.trainable_variables():
    tf.summary.histogram(v.name, v)
    tf.summary.histogram(v.name + '/gradients', tf.gradients(loss, v))

#measure_time([loss, unweighted_loss, acc_op, opt], n_times=10)

INFO:tensorflow:Summary name CNN/Conv1/Conv_dim10_ker32_pool2/weights:0 is illegal; using CNN/Conv1/Conv_dim10_ker32_pool2/weights_0 instead.
INFO:tensorflow:Summary name CNN/Conv1/Conv_dim10_ker32_pool2/weights:0/gradients is illegal; using CNN/Conv1/Conv_dim10_ker32_pool2/weights_0/gradients instead.
INFO:tensorflow:Summary name CNN/Conv1/Conv_dim10_ker32_pool2/BatchNorm/beta:0 is illegal; using CNN/Conv1/Conv_dim10_ker32_pool2/BatchNorm/beta_0 instead.
INFO:tensorflow:Summary name CNN/Conv1/Conv_dim10_ker32_pool2/BatchNorm/beta:0/gradients is illegal; using CNN/Conv1/Conv_dim10_ker32_pool2/BatchNorm/beta_0/gradients instead.
INFO:tensorflow:Summary name CNN/Conv2/Conv_dim10_ker32_pool2/weights:0 is illegal; using CNN/Conv2/Conv_dim10_ker32_pool2/weights_0 instead.
INFO:tensorflow:Summary name CNN/Conv2/Conv_dim10_ker32_pool2/weights:0/gradients is illegal; using CNN/Conv2/Conv_dim10_ker32_pool2/weights_0/gradients instead.
INFO:tensorflow:Summary name CNN/Conv2/Conv_dim10_ker32_pool

In [12]:
summaries = tf.summary.merge_all()

In [13]:
with tf.Session() as sess:
    print('Sess started')
    coord = tf.train.Coordinator()
    tf.global_variables_initializer().run()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    feed_dict={}
    init_state = sess.run(r.zero_state)
    rnn_feed = {r.init_state : init_state}
    feed_dict.update(rnn_feed)
    print('Evaluating')
    for i in range(10):
        t = time.time()
        test_output = sess.run([opt, loss, summaries], feed_dict)
        train_writer.add_summary(test_output[2], i)
        print(i, '/', 10, 'loss:', test_output[1])

    print('Closing threads')
    coord.request_stop()
    coord.join(threads)

Sess started
Evaluating
0 / 10 loss: 0.489348
1 / 10 loss: 0.418308
2 / 10 loss: 0.49641
3 / 10 loss: 0.473225
4 / 10 loss: 0.482493
5 / 10 loss: 0.362709
6 / 10 loss: 0.49135
7 / 10 loss: 0.406379
8 / 10 loss: 0.350535
9 / 10 loss: 0.336632
Closing threads
